In [1]:
import tkinter as tk
from tkinter import font  as tkfont
#from tkinter import *
from PIL import Image
from PIL import ImageTk
from tkinter import filedialog
import cv2
import os
import argparse
import dlib
import imutils
from scipy.spatial import distance as dist
from imutils.video import VideoStream, FPS
from imutils import face_utils
import imutils
import numpy as np
import time





class Assignment1(tk.Tk):

    def __init__(self, *args, **kwargs):
        tk.Tk.__init__(self, *args, **kwargs)

        self.title_font = tkfont.Font(family='Monospace', size=18, weight="bold")

        # Adding container
        container = tk.Frame(self)
        container.pack(side="top", fill="both", expand=True)
        container.grid_rowconfigure(0, weight=1)
        container.grid_columnconfigure(0, weight=1)

        self.frames = {}
        F=StartPage
        page_name = F.__name__
        frame = F(parent=container, controller=self)
        self.frames[page_name] = frame
        frame.grid(row=0, column=0, sticky="nsew")

        self.show_frame("StartPage")

    def show_frame(self, page_name):
        #Showing a frame for the given page name
        frame = self.frames[page_name]
        frame.tkraise()
        
    def eye_aspect_ratio(self,eye):
        A = dist.euclidean(eye[1], eye[5])
        B = dist.euclidean(eye[2], eye[4])
        C = dist.euclidean(eye[0], eye[3])
        ear = (A + B) / (2.0 * C)

        return ear
 
        
    def smile(self,mouth,shape):
        A = dist.euclidean(mouth[3], mouth[9])
        B = dist.euclidean(mouth[2], mouth[10])
        C = dist.euclidean(mouth[4], mouth[8])
        avg = (A+B+C)/3
        D = dist.euclidean(shape[34], shape[52])
        E = dist.euclidean(shape[34], shape[49])
        F = dist.euclidean(shape[34], shape[55])
        
        f1=E+F
        mar=D/f1
        return (mar)


    def smileplease(self):
        COUNTER = 0
        TOTAL = 0


        shape_predictor= "shape_predictor_68_face_landmarks.dat"
        detector = dlib.get_frontal_face_detector()
        predictor = dlib.shape_predictor(shape_predictor)


        (mStart, mEnd) = face_utils.FACIAL_LANDMARKS_IDXS["mouth"]

        print("[INFO] starting video stream thread...")
        vs = VideoStream(src=0).start()
        fileStream = False
        time.sleep(1.0)

        fps= FPS().start()
        #cv2.namedWindow("test")

        while True:
            frame = vs.read()
            frame = imutils.resize(frame, width=450)
            gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
            rects = detector(gray, 0)
            for rect in rects:
                shape = predictor(gray, rect)
                shape = face_utils.shape_to_np(shape)
                mouth= shape[mStart:mEnd]
                
                mar= self.smile(mouth,shape)
                mouthHull = cv2.convexHull(mouth)
        #print(shape)
                #cv2.drawContours(frame, [mouthHull], -1, (0, 255, 0), 1)
       

                if mar <= .17 :
                   COUNTER += 1
                   time.sleep(.5)
                   cv2.putText(frame, "Smile Detected", (30, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2)
                
                   if COUNTER >= 10:
                        TOTAL += 1
                        frame = vs.read()
                        time.sleep(.5)
                        frame2= frame.copy()
                        img_name = "opencv_frame_{}.png".format(TOTAL)
                        cv2.imwrite(img_name, frame)
                        print("{} written!".format(img_name))
    
                        COUNTER = 0
                else:
                  cv2.putText(frame, "Please smile", (30, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2)
                
                  #cv2.putText(frame, "MAR: {}".format(mar), (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2)

                cv2.imshow("Frame", frame)
                fps.update()

            key2 = cv2.waitKey(1) & 0xFF
            if key2 == ord('q'):
                break

            fps.stop()
            
    def SmartPassport(self):
        COUNTER = 0
        TOTAL = 0
        EYE_AR_THRESH = 0.3
        EYE_AR_CONSEC_FRAMES = 10
        COUNTER1=0


        shape_predictor= "shape_predictor_68_face_landmarks.dat"
        detector = dlib.get_frontal_face_detector()
        predictor = dlib.shape_predictor(shape_predictor)


        (mStart, mEnd) = face_utils.FACIAL_LANDMARKS_IDXS["mouth"]

        print("[INFO] starting video stream thread...")
        vs = VideoStream(src=0).start()
        fileStream = False
        time.sleep(1.0)

        fps= FPS().start()
        #cv2.namedWindow("test")

        while True:
            frame = vs.read()
            
            frame = imutils.resize(frame, width=450)
            image = cv2.rectangle(frame, (100, 50), (350,250), (36,255,12), 1)
            gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
            rects = detector(gray, 0)
            for rect in rects:
                shape = predictor(gray, rect)
                shape = face_utils.shape_to_np(shape)
                mouth= shape[mStart:mEnd]
                
                
                (lStart, lEnd) = face_utils.FACIAL_LANDMARKS_IDXS["left_eye"]
                (rStart, rEnd) = face_utils.FACIAL_LANDMARKS_IDXS["right_eye"]
                mar= self.smile(mouth,shape)
                mouthHull = cv2.convexHull(mouth)
                leftEye = shape[lStart:lEnd]
                rightEye = shape[rStart:rEnd]
                leftEAR = self.eye_aspect_ratio(leftEye)
                rightEAR = self.eye_aspect_ratio(rightEye)
                ear = (leftEAR + rightEAR) / 2.0
                leftEyeHull = cv2.convexHull(leftEye)
                rightEyeHull = cv2.convexHull(rightEye)
                ear = (leftEAR + rightEAR) / 2.0
              #cv2.drawContours(frame, [leftEyeHull], -1, (0, 255, 0), 1)
              #cv2.drawContours(frame, [rightEyeHull], -1, (0, 255, 0), 1)
        #print(shape)
                #cv2.drawContours(frame, [mouthHull], -1, (0, 255, 0), 1)
       

                if mar <= .17 or ear <=EYE_AR_THRESH :
                           COUNTER += 1
                           if COUNTER >= 10:
                                time.sleep(.3)
                                cv2.putText(frame, "Smile & eye close Detected!!! Please dont smile or close your eyes", (5, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.40, (0, 0, 255), 1)
                                time.sleep(.3)
                                COUNTER1 = 0
                                
                        
                else:
                    
                    cv2.putText(frame, "Keep your face inside the box & press space button to click", (30, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.40, (0, 0, 255), 1)
                    k = cv2.waitKey(1) 
                    if k == ord('q'):
                        break
                    elif k== 32:
                        img_name = "snapshot.png"
                        frame2= frame.copy()
                        cv2.imwrite(img_name, frame)
                        #cv2.imwrite(img_name, frame)
                        cv2.waitKey(10000)
                        image = cv2.imread("snapshot.png")
                        
                        crop_img = image[50:250, 100:350]
                        gamma=.8
                        lookUpTable = np.empty((1,256), np.uint8)
                        for i in range(256):
                              lookUpTable[0,i] = np.clip(pow(i / 255.0, gamma) * 255.0, 0, 255)
                        res = cv2.LUT(crop_img, lookUpTable)
                        
                        #brightness = 50
                        #contrast = 30
                        #img = np.int16(crop_img )
                        #img = img * (contrast/127+1) - contrast + brightness
                        #img = np.clip(img, 0, 255)
                        #img = np.uint8(img)
                        cv2.imshow("cropped", res)
                        cv2.waitKey(0)
                        
                    

               # cv2.putText(frame, "MAR: {}".format(mar), (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2)

                cv2.imshow("Frame", frame)
                fps.update()

            key2 = cv2.waitKey(1) & 0xFF
            if key2 == ord('q'):
                break

            fps.stop()
    
    def handpam(self):
        cap = cv2.VideoCapture(0)
        hand_cascade = cv2.CascadeClassifier('hand_haar_cascade.xml')
        HD=0
        HD2=0
        TOTAL=0
        COUNTER=20 
        while True:
                ret, frame = cap.read()
                
                blur = cv2.GaussianBlur(frame,(5,5),0)
                gray = cv2.cvtColor(blur, cv2.COLOR_BGR2GRAY)
                retval2,thresh1 = cv2.threshold(gray,70,255,cv2.THRESH_BINARY_INV+cv2.THRESH_OTSU)
                #print(thresh1)
                hand = hand_cascade.detectMultiScale(thresh1, 1.3, 5)  
                for (x,y,w,h) in hand:
                                time.sleep(.2)
                                cv2.putText(frame, "Hand detected timer Started", (30, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2)
                                HD=1
                                if HD==1:
                                    HD2=1
                                    break
                #print(HD2)
                if(HD2==1):
                    if COUNTER ==0:
                        TOTAL += 1
                        img_name = "opencv_frame_{}.png".format(TOTAL)
                        cv2.imwrite(img_name, frame)
                        print("{} written!".format(img_name))
                        HD=0
                        HD2=0
                        COUNTER=20
                    else:
                        COUNTER=COUNTER-1
                        print(COUNTER)
        
                  
            
                              
              
                                
                key2 = cv2.waitKey(1) & 0xFF
                if key2 == ord('q'):
                    break
                cv2.imshow("Frame", frame)
                
    
    
    
    
        cap.release()
        cv2.destroyAllWindows()
        
    
            
            


class StartPage(tk.Frame):
    

    def __init__(self, parent, controller):
        tk.Frame.__init__(self, parent)
        self.controller = controller
        label = tk.Label(self, text="SMART SELFIE", font=controller.title_font)
        label.pack(side="top", fill="x", pady=30)

        button1=tk.Button(self, text="Smile Please",command=lambda: controller.smileplease())
        button1.pack()
        button2=tk.Button(self, text="Smart Passport",command=lambda: controller.SmartPassport())
        button2.pack()
        button3=tk.Button(self, text="Palmcam",command=lambda: controller.handpam())
        button3.pack()
        
       
    


if __name__ == "__main__":
    
    app = Assignment1()
    app.mainloop()

[INFO] starting video stream thread...
[INFO] starting video stream thread...
opencv_frame_1.png written!
opencv_frame_2.png written!
19
18
17
16
15
14
13
12
11
10
9
8
7
6
5
4
3
2
1
0
opencv_frame_1.png written!
19
18
17
